In [ ]:
import os
import time
import google.generativeai as genai
from google.colab import userdata

# Get the Google api key. It is used for communication with Gemini in the chatbot
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# This function uploads files to Gemini
def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

# This function activates the chatbot when a file (study plan) is uploaded
def wait_for_files_active(files):
  """Waits for the given files to be active.

  Some files uploaded to the Gemini API need to be processed before they can be
  used as prompt inputs. The status can be seen by querying the file's "state"
  field.

  This implementation uses a simple blocking polling loop. Production code
  should probably employ a more sophisticated approach.
  """
  print("Waiting for file processing...")
  for name in (file.name for file in files):
    file = genai.get_file(name)
    while file.state.name == "PROCESSING":
      print(".", end="", flush=True)
      time.sleep(10)
      file = genai.get_file(name)
    if file.state.name != "ACTIVE":
      raise Exception(f"File {file.name} failed to process")
  print("...all files ready")
  print()

# Create the model
generation_config = {
  "temperature": 0.6,
  "top_p": 0.5,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

# Initialize the model
model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-lite-preview-02-05",
  generation_config=generation_config,
  system_instruction="You are an academic advisor at Abu Dhabi University. You help students enroll in courses, explain course materials, and suggest alternative courses. You are professional, polite, and friendly. You only answer questions related to academic advising.",
)

# TODO Make the file available on the local file system or in Colab
# You may need to update the file paths
files = [
  upload_to_gemini("BScSoftwareEngineeringStudyPlan2020", mime_type="application/pdf"),
]

# Some files have a processing delay. Wait for them to be ready.
wait_for_files_active(files)

# The chat history, retrieved from Google AI Studio
chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        files[0],
      ],
    },
  ]
)

# Interactive chatbot
def chat():
  print("\nChatbot is ready! type 'exit' to stop.")
  while True:
    user_input = input("\n------\nYou: ")
    if user_input.lower() == "exit":
      print("\nGoobye!")
      break

    # Send a message to Gemini and get a response
    response = chat_session.send_message(user_input)
    print("\nAcademic Advisor: ", response.text)


# Call the fucntion
chat()


Uploaded file 'BScSoftwareEngineeringStudyPlan2020 (5).pdf' as: https://generativelanguage.googleapis.com/v1beta/files/dikmh4cdosca
Waiting for file processing...
...all files ready


Chatbot is ready! type 'exit' to stop.

------
You: Hello!

Academic Advisor:  Hello! How can I help you today?


------
You: What is my major?

Academic Advisor:  Based on the study plan you provided, your major is Bachelor of Science in Software Engineering.


------
You: ما هو تخصصي؟

Academic Advisor:  بناءً على الخطة الدراسية التي قدمتها، تخصصك هو بكالوريوس العلوم في هندسة البرمجيات.


------
You: what courses should i take in my first semester?

Academic Advisor:  According to the study plan, in your first semester, you should take the following courses:

*   ENG200 - English II
*   FWS 100 - Academic Skills for Success
*   MTT 102 - Calculus I
*   STT 100 - General Statistics
*   ISL 100 - Islamic Culture


------
You: what courses should i take in my 3rd semester?

Academic Advisor:  In your third

KeyboardInterrupt: Interrupted by user